# 1. Installing and Importing Dependencies

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [21]:
import numpy as np
import pandas as pd

# 2. Instantiating Model

In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/Users/ayseljafarova/anaconda3/envs/newenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/Users/ayseljafarova/anaconda3/envs/newenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

# 3. Encoding and Calculating random Sentiment

In [3]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [4]:
result = model(tokens)

In [5]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [6]:
int(torch.argmax(result.logits))+1

4

# 4. Collecting Reviews from Trustpilot

In [89]:
urls = ["https://www.trustpilot.com/review/www.dugood.org",
       "https://www.trustpilot.com/review/www.dugood.org?page=2",
       "https://www.trustpilot.com/review/www.dugood.org?page=3",
       "https://www.trustpilot.com/review/www.dugood.org?page=4",
       "https://www.trustpilot.com/review/www.dugood.org?page=5",
       "https://www.trustpilot.com/review/www.dugood.org?page=6",
       "https://www.trustpilot.com/review/www.dugood.org?page=7",
       "https://www.trustpilot.com/review/www.dugood.org?page=8"]

reviews =[]

for url in urls: 
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    regex = re.compile('.*typography_color-black__5LYEn.*')
    results = soup.find_all('p', {'class':regex})
    review = [result.text for result in results]
    reviews.append(review)

#### Converting reviews list of list to a single list

In [90]:
reviews_single = [review for sublist in reviews for review in sublist]
reviews_single

['The customer service was amazing. The loan officer Allison Medina was exceptional and would recommend her to anyone. I have had other credit unions but this one beats them all. We were at Rave but gladly switched to here. Great service and awesome credit union and the employees that work here.',
 'I applied for an auto refinancing loan and it was processed the same day. Tracy at the Beaumont Eastex Freeway location was walked me through the process and was extremely helpful, friendly and professional. I couldn’t have been more pleased. ',
 'I financed my car through DuGood and I needed to have a account through them. Decided switching banks was easier then having multiple accounts open. Talked with Christy and she was very nice and helpful. She walked me through the whole process and explained everything in a simple and easy way to understand. I got more benefits then I did with my other bank and staff was so much nicer. Thank you Christy! Would recommend doing business here if you n

# 5. Loading Reviews into DataFrame and Scoring

In [91]:
df = pd.DataFrame(np.array(reviews_single), columns=['review'])

In [80]:
df['review'].iloc[2]

'I financed my car through DuGood and I needed to have a account through them. Decided switching banks was easier then having multiple accounts open. Talked with Christy and she was very nice and helpful. She walked me through the whole process and explained everything in a simple and easy way to understand. I got more benefits then I did with my other bank and staff was so much nicer. Thank you Christy! Would recommend doing business here if you need a good bank!'

#### Creating a function for sentiment calculation

In [92]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [82]:
sentiment_score(df['review'].iloc[1])

4

#### Calculating sentiment score of each reiew and loading newly created sentiments to the dataframe

In [93]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [88]:
df['sentiment'].iloc[56]

4

## Now we have a dataframe of 151 Reviews with their Sentiment Predictions as a column

In [94]:
df.shape

(151, 2)

In [95]:
df.head()

,review,sentiment
0,The customer service was amazing. The loan off...,5
1,I applied for an auto refinancing loan and it ...,4
2,I financed my car through DuGood and I needed ...,5
3,I was helped by an employee named Jada Needham...,4
4,My house is going to be back in the morning an...,3
